### Libraries

In [42]:
from torch.utils.data import DataLoader, random_split
from torch.optim import Adam, RMSprop

In [43]:
from data.preprocessing import *
from data.data_utils import *
from models.macro_architectures import *
from models.micro_architectures import *
from utils import *


# DATA

In [44]:
#format: YYYYMMDD
start_time = '20210101'
end_time = '20220801'
scrap_date = interval_time(start_time, end_time)

In [45]:
months = list(set([day[:6] for day in scrap_date]))
import_Dst(months)

In [47]:
l1_sample, l2_sample, dst, kp = automated_preprocessing(scrap_date)

100%|██████████| 274762/274762 [00:00<00:00, 333794.26it/s]


100%|██████████| 52216/52216 [00:00<00:00, 123728.75it/s]


100%|██████████| 185817647/185817647 [03:21<00:00, 922952.79it/s] 


100%|██████████| 51173/51173 [00:00<00:00, 124010.70it/s]


100%|██████████| 281064/281064 [00:01<00:00, 274350.04it/s]


...

100%|██████████| 183473529/183473529 [03:54<00:00, 783276.55it/s] 


100%|██████████| 53222/53222 [00:00<00:00, 128782.79it/s]


100%|██████████| 297497/297497 [00:00<00:00, 362743.24it/s]


100%|██████████| 51818/51818 [00:00<00:00, 125165.32it/s]


 14%|█▍        | 25952256/180838393 [00:44<07:08, 361218.38it/s] 

### Solving missing values with interpolation

In [8]:
start_time_ = f'{start_time[:4]}-{start_time[4:6]}-{start_time[-2:]} 00:00:00'
end_time_ = f'{end_time[:4]}-{end_time[4:6]}-{end_time[-2:]} 23:59:00' 

In [9]:
freq = '1T'

full_time_index = pd.date_range(start=start_time_, end=end_time_, freq=freq)

l1_sample = l1_sample.reindex(full_time_index)

l1_sample = l1_sample.interpolate(method='linear')

In [10]:
l2_sample = l2_sample.reindex(full_time_index)

l2_sample = l2_sample.interpolate(method='linear')

# L1 (raw) data

## Minute based

In [11]:
l1_sample

,proton_vx_gse,proton_vy_gse,proton_vz_gse,proton_vx_gsm,proton_vy_gsm,proton_vz_gsm,proton_speed,proton_density,proton_temperature,bt,bx_gse,by_gse,bz_gse,theta_gse,phi_gse,bx_gsm,by_gsm,bz_gsm,theta_gsm,phi_gsm
2022-08-01 00:00:00,-478.65714,43.071430,-18.400000,-478.65714,40.957146,-22.714285,480.95712,8.510000,173441.860,8.791893,7.086200,-4.019755,3.270043,21.840690,330.44970,7.086200,-3.664647,3.663627,24.632782,332.67020
2022-08-01 00:01:00,-480.98570,42.057140,-22.585714,-480.98570,39.514286,-26.742857,483.37143,8.464286,185467.140,8.582591,6.739781,-4.466785,2.769711,18.872953,326.47080,6.739781,-4.159703,3.212529,22.036581,328.33280
2022-08-01 00:02:00,-491.07144,56.200000,-22.014284,-491.07144,53.642857,-27.642857,495.05713,9.305715,218374.280,7.703689,5.968021,-2.387868,3.884053,30.909138,339.15740,5.968021,-1.976642,4.108515,32.785530,314.55447
2022-08-01 00:03:00,-493.52856,68.014290,-24.000000,-493.52856,65.214290,-30.857143,498.91428,9.671428,228698.580,7.239092,5.971862,-0.815659,3.879902,32.538246,309.05472,5.971862,-0.412276,3.943251,33.105440,263.87167
2022-08-01 00:04:00,-484.83334,53.383330,-15.783334,-484.83334,51.483334,-21.199999,488.08334,9.080000,204210.000,7.844100,6.698265,-1.942467,3.496836,26.688057,343.98862,6.698265,-1.571605,3.678486,28.167646,346.98236
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-08-01 23:55:00,-390.41000,20.689999,-2.420000,-390.41000,20.330000,-4.620000,390.99000,8.613000,79090.400,8.708879,3.890249,-7.049568,3.313216,22.363012,298.89180,3.890249,-6.658564,4.041955,27.656048,300.29570
2023-08-01 23:56:00,-393.74445,17.411110,-2.655556,-393.74445,17.011112,-4.477778,394.14444,9.308888,104618.664,8.466495,3.683012,-7.056999,2.848440,19.695435,297.55127,3.683012,-6.714469,3.581986,25.071960,298.73755
2023-08-01 23:57:00,-390.28183,17.427273,-3.790909,-390.28183,16.936363,-5.627273,390.70908,8.784545,78793.630,8.379578,3.853854,-6.783406,3.052517,21.368494,299.60187,3.853854,-6.419947,3.757228,26.645920,300.97607
2023-08-01 23:58:00,-391.43634,20.827272,2.209091,-391.43634,20.954546,-0.027273,392.16360,9.211819,100566.910,8.261917,4.074574,-4.820900,4.966876,37.815850,311.42163,4.074574,-4.263566,5.452666,42.207394,315.36426


## Hour based

In [12]:
l1_sample_hour = l1_sample.resample('60min').mean()
l1_sample_hour

,proton_vx_gse,proton_vy_gse,proton_vz_gse,proton_vx_gsm,proton_vy_gsm,proton_vz_gsm,proton_speed,proton_density,proton_temperature,bt,bx_gse,by_gse,bz_gse,theta_gse,phi_gse,bx_gsm,by_gsm,bz_gsm,theta_gsm,phi_gsm
2022-08-01 00:00:00,-489.302281,57.179857,-21.269801,-489.302281,54.485076,-27.490186,493.251960,9.247633,198485.987167,7.543399,6.291421,-1.856414,2.859877,22.954698,300.997411,6.291421,-1.527907,3.041643,24.361645,297.380635
2022-08-01 01:00:00,-508.790815,51.889845,-27.850028,-508.790815,47.635167,-34.453706,512.453934,9.912636,234197.626667,6.141733,4.452409,-2.402331,1.739807,16.432656,265.105190,4.452409,-2.157465,2.061328,19.703389,240.259886
2022-08-01 02:00:00,-522.989989,46.409234,-34.373814,-522.989989,40.170385,-41.611131,526.401869,9.133113,218915.152667,5.931544,0.184564,-2.469814,-0.172089,-2.568171,204.232442,0.184564,-2.453588,0.207707,1.573937,197.414516
2022-08-01 03:00:00,-498.359199,43.835917,-35.477302,-498.359199,35.712523,-43.524897,501.855898,7.695283,193808.512500,6.292952,5.306359,-1.672818,1.302133,11.992297,274.047466,5.306359,-1.384970,1.622839,15.157878,257.554610
2022-08-01 04:00:00,-525.964391,31.554381,-37.966738,-525.964391,21.319488,-44.454766,528.562193,8.653108,209614.043500,5.501450,0.355088,-3.930183,-0.112578,-1.028107,266.247303,0.355088,-3.853669,0.840545,10.024766,266.188024
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-08-01 19:00:00,-382.680769,0.217665,-15.192104,-382.680769,-2.215317,-15.036393,383.136132,9.807184,66666.911150,9.816390,6.217887,-7.306248,-0.344754,-2.062787,310.312771,6.217887,-7.258762,0.820913,4.873037,310.610893
2023-08-01 20:00:00,-382.508196,-0.425859,-19.921152,-382.508196,-2.982734,-19.695648,383.090436,10.251275,56155.322900,9.312197,5.133512,-7.576634,-1.271946,-7.821866,304.145239,5.133512,-7.679039,-0.282955,-1.693841,303.807800
2023-08-01 21:00:00,-378.749960,-7.459660,-28.139573,-378.749960,-10.480231,-27.167887,380.014618,9.695502,58283.928467,8.919270,5.942875,-6.403760,-0.588859,-3.800311,312.912670,5.942875,-6.430948,0.113013,0.756934,312.845110
2023-08-01 22:00:00,-388.897926,9.909498,-4.975384,-388.897926,9.369652,-5.926781,389.483691,9.748028,103017.672383,8.472186,5.815027,-5.407756,2.184644,14.794651,316.942325,5.815027,-5.162399,2.712489,18.609071,318.306821


# L2 (cleaned) data

## Minute based

In [13]:
l2_sample

,proton_vx_gse,proton_vy_gse,proton_vz_gse,proton_vx_gsm,proton_vy_gsm,proton_vz_gsm,proton_speed,proton_density,proton_temperature,bt,bx_gse,by_gse,bz_gse,theta_gse,phi_gse,bx_gsm,by_gsm,bz_gsm,theta_gsm,phi_gsm
2022-08-01 00:00:00,-478.8,42.3,-18.2,-478.8,40.3,-22.5,481.0,8.43,173700.0,8.79,7.08,-4.03,3.28,21.91,330.34,7.08,-3.68,3.67,24.72,332.56
2022-08-01 00:01:00,-480.9,42.2,-23.3,-480.9,39.6,-27.5,483.3,8.40,180724.0,8.59,6.74,-4.48,2.76,18.81,326.42,6.74,-4.17,3.20,21.98,328.26
2022-08-01 00:02:00,-491.4,56.1,-22.1,-491.4,53.6,-27.7,495.1,9.23,209072.0,7.69,6.04,-2.46,3.86,30.63,337.82,6.04,-2.05,4.10,32.69,341.22
2022-08-01 00:03:00,-493.5,67.6,-25.9,-493.5,64.6,-32.7,498.8,9.64,225575.0,7.24,5.99,-0.88,3.86,32.52,351.67,5.99,-0.48,3.93,33.18,355.46
2022-08-01 00:04:00,-483.7,52.9,-15.9,-483.7,51.0,-21.2,486.9,9.06,200746.0,7.87,6.75,-1.88,3.51,26.62,344.43,6.75,-1.51,3.69,28.06,347.39
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-08-01 23:55:00,-390.4,19.9,-3.1,-390.4,19.5,-5.2,390.9,8.57,79088.0,8.71,3.90,-7.05,3.33,22.46,298.92,3.90,-6.66,4.06,27.75,300.33
2023-08-01 23:56:00,-394.4,17.7,-2.6,-394.4,17.3,-4.5,394.8,9.41,110712.0,8.46,3.67,-7.05,2.86,19.79,297.53,3.67,-6.71,3.59,25.17,298.72
2023-08-01 23:57:00,-389.5,17.7,-4.3,-389.5,17.1,-6.2,389.9,8.63,58271.0,8.38,3.85,-6.78,3.05,21.38,299.62,3.85,-6.42,3.76,26.65,300.99
2023-08-01 23:58:00,-391.6,20.7,2.4,-391.6,20.8,0.2,392.2,9.19,108681.0,8.26,4.07,-5.07,4.69,35.80,308.73,4.07,-4.55,5.21,40.47,311.84


## Hour based

In [14]:
l2_sample_hour = l2_sample.resample('60min').mean()
l2_sample_hour

,proton_vx_gse,proton_vy_gse,proton_vz_gse,proton_vx_gsm,proton_vy_gsm,proton_vz_gsm,proton_speed,proton_density,proton_temperature,bt,bx_gse,by_gse,bz_gse,theta_gse,phi_gse,bx_gsm,by_gsm,bz_gsm,theta_gsm,phi_gsm
2022-08-01 00:00:00,-489.208333,57.145000,-21.240000,-489.208333,54.460000,-27.443333,493.095000,9.246667,197578.583333,7.543333,6.303667,-1.872000,2.862500,23.042667,303.207333,6.303667,-1.544167,3.046000,24.468000,305.825500
2022-08-01 01:00:00,-508.771667,51.758333,-27.611667,-508.771667,47.553333,-34.195000,512.376667,9.911833,234221.300000,6.146833,4.458000,-2.411000,1.750000,16.625167,259.985167,4.458000,-2.165167,2.072333,19.910000,244.457500
2022-08-01 02:00:00,-522.971667,46.445000,-34.198333,-522.971667,40.241667,-41.433333,526.340000,9.119667,217786.800000,5.940000,0.200833,-2.490000,-0.181167,-2.719167,202.435000,0.200833,-2.474333,0.201833,1.558500,193.670667
2022-08-01 03:00:00,-498.388333,43.578333,-35.293333,-498.388333,35.510000,-43.275000,501.780000,7.701000,193128.650000,6.293667,5.318833,-1.675667,1.297167,12.009167,270.250167,5.318833,-1.389833,1.618000,15.186333,242.839500
2022-08-01 04:00:00,-526.130000,31.710000,-37.916667,-526.130000,21.496667,-44.436667,528.681667,8.655500,209349.483333,5.515833,0.348000,-3.937167,-0.106500,-1.025167,267.512000,0.348000,-3.859000,0.846833,10.312333,267.249167
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-08-01 19:00:00,-382.586667,0.195000,-15.218333,-382.586667,-2.250000,-15.053333,382.978333,9.779667,65301.266667,9.817000,6.212000,-7.315667,-0.357333,-2.134333,310.247500,6.212000,-7.269500,0.811833,4.826333,310.531000
2023-08-01 20:00:00,-382.468333,-0.383333,-19.765000,-382.468333,-2.926667,-19.533333,383.011667,10.239167,55382.900000,9.318000,5.136667,-7.581167,-1.270667,-7.814667,304.145667,5.136667,-7.683833,-0.279333,-1.674333,303.811667
2023-08-01 21:00:00,-378.730000,-7.340000,-28.055000,-378.730000,-10.360000,-27.098333,379.943333,9.687167,57833.183333,8.919167,5.942833,-6.407333,-0.580333,-3.743667,312.893000,5.942833,-6.433500,0.122500,0.823333,312.825667
2023-08-01 22:00:00,-388.971667,10.000000,-4.993333,-388.971667,9.453333,-5.953333,389.476667,9.755667,102499.000000,8.478000,5.823167,-5.413333,2.193833,14.888167,316.923833,5.823167,-5.166667,2.723000,18.712333,318.278333


# Dst data

In [15]:
dst

0        4
1        8
2       10
3        3
4       -1
        ..
8779    14
8780    13
8781    11
8782     6
8783     5
Name: Dst, Length: 8784, dtype: int64

# Kp data

In [16]:
kp

0       3-
1       2+
2       2-
3       2-
4        2
        ..
2923    1+
2924     3
2925    3-
2926    2+
2927     2
Name: Kp, Length: 2928, dtype: object

# Datasets
### Descriptions:
**hn_dl**: hour normal dataloader

**mn_dl**: minute normal dataloader

**hr_dl**: minute normal dataloader

**mr_dl**: minute normal dataloader



In [17]:
device = get_default_device()

In [18]:
sequence_length_hour = 10  #hour
sequence_length_minute = 600 #minute
pred_length = 6 #hours

In [19]:
##Normal
hour_Normal_dataset = NormalTrainingDataset(l1_sample_hour, dst, kp, sequence_length_hour, pred_length, hour = True)
minute_Normal_dataset = NormalTrainingDataset(l1_sample, dst, kp, sequence_length_minute, pred_length, hour = False)
##Refined(new method)
hour_Refined_dataset = RefinedTrainingDataset(l1_sample_hour, l2_sample_hour, dst,kp,sequence_length_hour, pred_length, hour = True)
minute_Refined_dataset = RefinedTrainingDataset(l1_sample, l2_sample, dst,kp,sequence_length_minute, pred_length, hour = False)

In [20]:
#Test:15% training: 85%

test_size = round(0.15*len(hour_Normal_dataset))

train_hn_ds, test_hn_ds = random_split(hour_Normal_dataset , [len(hour_Normal_dataset) - test_size, test_size])

batch_size = 32  #Change based on GPU capacity

train_hn_dl = DataLoader(train_hn_ds, batch_size, shuffle=True, num_workers=4, pin_memory=True)
train_hn_dl = DeviceDataLoader(train_hn_dl, device)
test_hn_dl = DataLoader(test_hn_ds, batch_size*2, num_workers=4, pin_memory=True)
test_hn_dl = DeviceDataLoader(test_hn_dl, device)

In [21]:
#Test:15% training: 85%

test_size = round(0.15*len(minute_Normal_dataset))

train_mn_ds, test_mn_ds = random_split(minute_Normal_dataset , [len(minute_Normal_dataset) - test_size, test_size])

batch_size = 32  #Change based on GPU capacity

train_mn_dl = DataLoader(train_mn_ds, batch_size, shuffle=True, num_workers=4, pin_memory=True)
train_mn_dl = DeviceDataLoader(train_mn_dl, device)
test_mn_dl = DataLoader(test_mn_ds, batch_size*2, num_workers=4, pin_memory=True)
test_mn_dl = DeviceDataLoader(test_mn_dl, device)


In [22]:
#Test:15% training: 85%

test_size = round(0.15*len(hour_Refined_dataset))

train_hr_ds, test_hr_ds = random_split(hour_Refined_dataset , [len(hour_Refined_dataset) - test_size, test_size])

batch_size = 32  #Change based on GPU capacity

train_hr_dl = DataLoader(train_hr_ds, batch_size, shuffle=True, num_workers=4, pin_memory=True)
train_hr_dl = DeviceDataLoader(train_hr_dl, device)
test_hr_dl = DataLoader(test_hr_ds, batch_size*2, num_workers=4, pin_memory=True)
test_hr_dl = DeviceDataLoader(test_hr_dl, device)

In [23]:
#Test:15% training: 85%

test_size = round(0.15*len(minute_Refined_dataset))

train_mr_ds, test_mr_ds = random_split(minute_Refined_dataset , [len(minute_Refined_dataset) - test_size, test_size])

batch_size = 32  #Change based on GPU capacity

train_mr_dl = DataLoader(train_mr_ds, batch_size, shuffle=True, num_workers=4, pin_memory=True)
train_mr_dl = DeviceDataLoader(train_mr_dl, device)
test_mr_dl = DataLoader(test_mr_ds, batch_size*2, num_workers=4, pin_memory=True)
test_mr_dl = DeviceDataLoader(test_mr_dl, device)

# Models

## Refined models with attention

### 1d Convolutional encoder

In [24]:
## Architecture
hidden_size = 10
input_size = 20
##nn architecture
architecture = (10,10,10)

In [25]:
#encoders
cnn_encoder_min = Simple1DCNN(architecture, sequence_length_minute, hidden_size)
cnn_encoder_hour = Simple1DCNN(architecture, sequence_length_hour, hidden_size)
#fc layer
cnn_fc_dst_min = DeepNeuralNetwork(hidden_size, pred_length, *architecture)
cnn_fc_kp_min = DeepNeuralNetwork(hidden_size, hour_to_3_hour(pred_length), *architecture)

cnn_fc_dst_hour = DeepNeuralNetwork(hidden_size, pred_length, *architecture)
cnn_fc_kp_hour = DeepNeuralNetwork(hidden_size, hour_to_3_hour(pred_length), *architecture)

In [26]:
RefinedCNN_min = to_device(RefinedArchitecture(cnn_encoder_min, cnn_fc_dst_min, cnn_fc_kp_min), device)
RefinedCNN_hour = to_device(RefinedArchitecture(cnn_encoder_hour, cnn_fc_dst_hour, cnn_fc_kp_hour), device)

In [27]:
##hyperparameters
epochs = 20
max_lr = 5e-4
weigth_decay = 1e-6
grad_clip = 1e-1
opt_func = Adam
#opt_func = RMSprop

In [28]:
RefinedCNN_min_history = []

In [29]:
RefinedCNN_min_history += RefinedCNN_min.fit(epochs, max_lr, train_mr_dl, test_mr_dl, weigth_decay, grad_clip, opt_func)

KeyboardInterrupt: 

In [ ]:
RefinedCNN_hour_history = []

In [ ]:
RefinedCNN_hour_history += RefinedCNN_hour.fit(epochs, max_lr, train_hr_dl, test_hr_dl, weigth_decay, grad_clip, opt_func)

Epoch [0]
	last_lr: 0.00005
	train_overall_loss: 2.0356
	train_main_loss: 2.0356
	train_output_loss: 0.0000
	train_encoder_loss: 0.0000
	val_overall_loss: 2.2384
	val_main_loss: 2.2384
	val_output_loss: 0.0000
	val_encoder_loss: 0.0000
Epoch [1]
	last_lr: 0.00014
	train_overall_loss: 2.0138
	train_main_loss: 2.0138
	train_output_loss: 0.0000
	train_encoder_loss: 0.0000
	val_overall_loss: 2.2065
	val_main_loss: 2.2065
	val_output_loss: 0.0000
	val_encoder_loss: 0.0000
Epoch [2]
	last_lr: 0.00026
	train_overall_loss: 1.9698
	train_main_loss: 1.9698
	train_output_loss: 0.0000
	train_encoder_loss: 0.0000
	val_overall_loss: 2.0833
	val_main_loss: 2.0833
	val_output_loss: 0.0000
	val_encoder_loss: 0.0002
Epoch [3]
	last_lr: 0.00038
	train_overall_loss: 1.5331
	train_main_loss: 1.5331
	train_output_loss: 0.0002
	train_encoder_loss: 0.0016
	val_overall_loss: 1.5814
	val_main_loss: 1.5814
	val_output_loss: 0.0003
	val_encoder_loss: 0.0013
Epoch [4]
	last_lr: 0.00047
	train_overall_loss: 1.3133


KeyboardInterrupt: 

### Bidirectional Deep LSTM encoder

In [ ]:
## Architecture
hidden_size = 10
input_size = 20
##nn architecture
architecture = ()

In [ ]:
#encoders
##Bidirectional minute based models
deep_lstm_encoder_min_forward = DeepLSTM(hidden_size, input_size, architecture)
deep_lstm_encoder_min_backward = DeepLSTM(hidden_size, input_size, architecture)
bidirectional_deeplstm_encoder_min = BidirectionalRNNWithAttention(deep_lstm_encoder_min_forward, deep_lstm_encoder_min_backward)
##Bidirectional hour based models
deep_lstm_encoder_hour_forward = DeepLSTM(hidden_size, input_size, architecture)
deep_lstm_encoder_hour_backward = DeepLSTM(hidden_size, input_size, architecture)
bidirectional_deeplstm_encoder_hour = BidirectionalRNNWithAttention(deep_lstm_encoder_hour_forward, deep_lstm_encoder_hour_backward)
#fc layer
deep_lstm_fc_dst_min = DeepNeuralNetwork(2*hidden_size, pred_length, *architecture) #the multiplying factor because concatenating hidden_states on bidirectional arch
deep_lstm_fc_kp_min = DeepNeuralNetwork(2*hidden_size, hour_to_3_hour(pred_length), *architecture)

deep_lstm_fc_dst_hour = DeepNeuralNetwork(2*hidden_size, pred_length, *architecture)
deep_lstm_fc_kp_hour = DeepNeuralNetwork(2*hidden_size, hour_to_3_hour(pred_length), *architecture)

In [ ]:
RefinedLSTM_min = to_device(RefinedArchitecture(bidirectional_deeplstm_encoder_min, deep_lstm_fc_dst_min, deep_lstm_fc_kp_min), device)
RefinedLSTM_hour = to_device(RefinedArchitecture(bidirectional_deeplstm_encoder_hour, deep_lstm_fc_dst_hour, deep_lstm_fc_kp_hour), device)

In [ ]:
##hyperparameters
epochs = 100
max_lr = 1e-3
weigth_decay = 1e-4
grad_clip = 1e-2
#opt_func = Adam
opt_func = RMSprop

In [ ]:
RefinedLSTM_min_history = []

In [ ]:
RefinedLSTM_min_history += RefinedLSTM_min.fit(epochs, max_lr, train_mr_dl, test_mr_dl, weigth_decay, grad_clip, opt_func)

KeyboardInterrupt: 

In [ ]:
RefinedLSTM_hour_history = []

In [ ]:
RefinedLSTM_hour_history += RefinedLSTM_hour.fit(epochs, max_lr, train_hr_dl, test_hr_dl, weigth_decay, grad_clip, opt_func)

Epoch [0]
	last_lr: 0.00004
	train_overall_loss: 1.3423
	train_main_loss: 1.3423
	train_output_loss: 0.0002
	train_encoder_loss: 0.0000
	val_overall_loss: 0.9733
	val_main_loss: 0.9733
	val_output_loss: 0.0002
	val_encoder_loss: 0.0000
Epoch [1]
	last_lr: 0.00005
	train_overall_loss: 0.9529
	train_main_loss: 0.9529
	train_output_loss: 0.0005
	train_encoder_loss: 0.0001
	val_overall_loss: 0.8512
	val_main_loss: 0.8512
	val_output_loss: 0.0004
	val_encoder_loss: 0.0001
Epoch [2]
	last_lr: 0.00006
	train_overall_loss: 0.8301
	train_main_loss: 0.8301
	train_output_loss: 0.0008
	train_encoder_loss: 0.0001
	val_overall_loss: 0.7807
	val_main_loss: 0.7807
	val_output_loss: 0.0004
	val_encoder_loss: 0.0001
Epoch [3]
	last_lr: 0.00008
	train_overall_loss: 0.7586
	train_main_loss: 0.7586
	train_output_loss: 0.0007
	train_encoder_loss: 0.0001
	val_overall_loss: 0.7456
	val_main_loss: 0.7456
	val_output_loss: 0.0005
	val_encoder_loss: 0.0001
Epoch [4]
	last_lr: 0.00010
	train_overall_loss: 0.7159


KeyboardInterrupt: 

### Bidirectional Deep GRU encoder

In [ ]:
## Architecture
hidden_size = 10
input_size = 20
##nn architecture
architecture = ()

In [ ]:
#encoders
##Bidirectional minute based models
deep_gru_encoder_min_forward= DeepGRU(hidden_size, input_size, architecture)
deep_gru_encoder_min_backward= DeepGRU(hidden_size, input_size, architecture)
bidirectional_deepgru_encoder_min = BidirectionalRNNWithAttention(deep_gru_encoder_min_forward, deep_gru_encoder_min_backward)
##Bidirectional hour based models
deep_gru_encoder_hour_forward = DeepGRU(hidden_size, input_size, architecture)
deep_gru_encoder_hour_backward = DeepGRU(hidden_size, input_size, architecture)
bidirectional_deepgru_encoder_hour = BidirectionalRNNWithAttention(deep_gru_encoder_hour_forward, deep_gru_encoder_hour_backward)
#fc layer
deep_gru_fc_dst_min = DeepNeuralNetwork(2*hidden_size, pred_length, *architecture)
deep_gru_fc_kp_min = DeepNeuralNetwork(2*hidden_size, hour_to_3_hour(pred_length), *architecture)

deep_gru_fc_dst_hour = DeepNeuralNetwork(2*hidden_size, pred_length, *architecture)
deep_gru_fc_kp_hour = DeepNeuralNetwork(2*hidden_size, hour_to_3_hour(pred_length), *architecture)

In [ ]:
RefinedGRU_min = to_device(RefinedArchitecture(bidirectional_deepgru_encoder_min, deep_gru_fc_dst_min, deep_gru_fc_kp_min), device)
RefinedGRU_hour = to_device(RefinedArchitecture(bidirectional_deepgru_encoder_hour, deep_gru_fc_dst_hour, deep_gru_fc_kp_hour), device)

In [ ]:
##hyperparameters
epochs = 100
max_lr = 1e-2
weigth_decay = 1e-6
grad_clip = 5e-1
opt_func = Adam
#opt_func = RMSprop

In [ ]:
RefinedGRU_min_history = []

In [ ]:
RefinedGRU_min_history += RefinedGRU_min.fit(epochs, max_lr, train_mr_dl, test_mr_dl, weigth_decay, grad_clip, opt_func)

In [ ]:
RefinedGRU_hour_history = []

In [ ]:
RefinedGRU_hour_history += RefinedGRU_hour.fit(epochs, max_lr, train_hr_dl, test_hr_dl, weigth_decay, grad_clip, opt_func)

Epoch [0]
	last_lr: 0.00043
	train_overall_loss: 1.6406
	train_main_loss: 1.6406
	train_output_loss: 0.0001
	train_encoder_loss: 0.0001
	val_overall_loss: 1.4343
	val_main_loss: 1.4343
	val_output_loss: 0.0002
	val_encoder_loss: 0.0001
Epoch [1]
	last_lr: 0.00050
	train_overall_loss: 1.1422
	train_main_loss: 1.1422
	train_output_loss: 0.0002
	train_encoder_loss: 0.0001
	val_overall_loss: 1.1519
	val_main_loss: 1.1519
	val_output_loss: 0.0003
	val_encoder_loss: 0.0001


KeyboardInterrupt: 

### Bidirectional Deep Vanilla RNN encoder

In [ ]:
## Architecture
hidden_size = 10
input_size = 20
##nn architecture
architecture = ()

In [ ]:
#encoders
##Bidirectional minute encoders
deep_rnn_encoder_min_forward = DeepVanillaRNN(hidden_size, input_size, architecture)
deep_rnn_encoder_min_backward = DeepVanillaRNN(hidden_size, input_size, architecture)
bidirectional_deeprnn_encoder_min = BidirectionalRNNWithAttention(deep_rnn_encoder_min_forward,deep_rnn_encoder_min_backward)
##Bidirectional hour encoders
deep_rnn_encoder_hour_forward = DeepVanillaRNN(hidden_size, input_size, architecture)
deep_rnn_encoder_hour_backward = DeepVanillaRNN(hidden_size, input_size, architecture)
bidirectional_deeprnn_encoder_hour = BidirectionalRNNWithAttention(deep_rnn_encoder_hour_forward,deep_rnn_encoder_hour_backward)
#fc layer
deep_rnn_fc_dst_min = DeepNeuralNetwork(2*hidden_size, pred_length, *architecture)
deep_rnn_fc_kp_min = DeepNeuralNetwork(2*hidden_size, hour_to_3_hour(pred_length), *architecture)

deep_rnn_fc_dst_hour = DeepNeuralNetwork(2*hidden_size, pred_length, *architecture)
deep_rnn_fc_kp_hour = DeepNeuralNetwork(2*hidden_size, hour_to_3_hour(pred_length), *architecture)

In [ ]:
RefinedVanillaRNN_min = to_device(RefinedArchitecture(bidirectional_deeprnn_encoder_min, deep_rnn_fc_dst_min, deep_rnn_fc_kp_min), device)
RefinedVanillaRNN_hour = to_device(RefinedArchitecture(bidirectional_deeprnn_encoder_hour, deep_rnn_fc_dst_hour, deep_rnn_fc_kp_hour), device)

In [ ]:
##hyperparameters
epochs = 100
max_lr = 1e-3
weigth_decay = 1e-4
grad_clip = 1e-2
#opt_func = Adam
opt_func = RMSprop

In [ ]:
RefinedVanillaRNN_min_history = []

In [ ]:
RefinedVanillaRNN_min_history += RefinedVanillaRNN_min.fit(epochs, max_lr, train_mr_dl, test_mr_dl, weigth_decay, grad_clip, opt_func)

In [ ]:
RefinedVanillaRNN_hour_history = []

In [ ]:
RefinedVanillaRNN_hour_history += RefinedVanillaRNN_hour.fit(epochs, max_lr, train_hr_dl, test_hr_dl, weigth_decay, grad_clip, opt_func)

Epoch [0]
	last_lr: 0.00004
	train_overall_loss: 1.3494
	train_main_loss: 1.3494
	train_output_loss: 0.0001
	train_encoder_loss: 0.0001
	val_overall_loss: 1.2106
	val_main_loss: 1.2106
	val_output_loss: 0.0002
	val_encoder_loss: 0.0001
Epoch [1]
	last_lr: 0.00005
	train_overall_loss: 1.0410
	train_main_loss: 1.0410
	train_output_loss: 0.0003
	train_encoder_loss: 0.0001
	val_overall_loss: 1.0773
	val_main_loss: 1.0773
	val_output_loss: 0.0004
	val_encoder_loss: 0.0001


KeyboardInterrupt: 

### Bidirectional non deep architectures

#### LSTM

In [ ]:
non_deep_lstm_forward_min = LSTM(input_size, hidden_size)
non_deep_lstm_backward_min = LSTM(input_size, hidden_size)
bidirectional_lstm_encoder_min = BidirectionalRNNWithAttention(non_deep_lstm_forward_min, non_deep_lstm_backward_min)

non_deep_lstm_forward_hour = LSTM(input_size, hidden_size)
non_deep_lstm_backward_hour = LSTM(input_size, hidden_size)
bidirectional_lstm_encoder_hour = BidirectionalRNNWithAttention(non_deep_lstm_forward_hour, non_deep_lstm_backward_hour)
#fc layer
non_deep_lstm_fc_dst_min = DeepNeuralNetwork(2*hidden_size, pred_length, *architecture)
non_deep_lstm_fc_kp_min = DeepNeuralNetwork(2*hidden_size, hour_to_3_hour(pred_length), *architecture)

non_deep_lstm_fc_dst_hour = DeepNeuralNetwork(2*hidden_size, pred_length, *architecture)
non_deep_lstm_fc_kp_hour = DeepNeuralNetwork(2*hidden_size, hour_to_3_hour(pred_length), *architecture)

In [ ]:
RefinedNonDeepLSTM_min = to_device(RefinedArchitecture(bidirectional_lstm_encoder_min, non_deep_lstm_fc_dst_min, non_deep_lstm_fc_kp_min), device)
RefinedNonDeepLSTM_hour = to_device(RefinedArchitecture(bidirectional_lstm_encoder_hour, non_deep_lstm_fc_dst_hour, non_deep_lstm_fc_kp_hour), device)

In [ ]:
RefinedNonDeepLSTM_min_history = []

In [ ]:
RefinedNonDeepLSTM_min_history += RefinedNonDeepLSTM_min.fit(epochs, max_lr, train_mr_dl, test_mr_dl, weigth_decay, grad_clip, opt_func)

In [ ]:
RefinedNonDeepLSTM_hour_history = []

In [ ]:
RefinedNonDeepLSTM_hour_history += RefinedNonDeepLSTM_hour.fit(epochs, max_lr, train_hr_dl, test_hr_dl, weigth_decay, grad_clip, opt_func)

Epoch [0]
	last_lr: 0.00004
	train_overall_loss: 1.3338
	train_main_loss: 1.3338
	train_output_loss: 0.0002
	train_encoder_loss: 0.0001
	val_overall_loss: 1.1319
	val_main_loss: 1.1319
	val_output_loss: 0.0004
	val_encoder_loss: 0.0001
Epoch [1]
	last_lr: 0.00005
	train_overall_loss: 0.9605
	train_main_loss: 0.9605
	train_output_loss: 0.0005
	train_encoder_loss: 0.0001
	val_overall_loss: 0.9760
	val_main_loss: 0.9760
	val_output_loss: 0.0005
	val_encoder_loss: 0.0001
Epoch [2]
	last_lr: 0.00006
	train_overall_loss: 0.8491
	train_main_loss: 0.8491
	train_output_loss: 0.0008
	train_encoder_loss: 0.0002
	val_overall_loss: 0.8663
	val_main_loss: 0.8663
	val_output_loss: 0.0005
	val_encoder_loss: 0.0001
Epoch [3]
	last_lr: 0.00008
	train_overall_loss: 0.7699
	train_main_loss: 0.7698
	train_output_loss: 0.0008
	train_encoder_loss: 0.0002
	val_overall_loss: 0.7792
	val_main_loss: 0.7792
	val_output_loss: 0.0006
	val_encoder_loss: 0.0002
Epoch [4]
	last_lr: 0.00010
	train_overall_loss: 0.7140


KeyboardInterrupt: 

#### GRU

In [ ]:
non_deep_gru_forward_min = GRU(input_size, hidden_size)
non_deep_gru_backward_min = GRU(input_size, hidden_size)
bidirectional_gru_encoder_min = BidirectionalRNNWithAttention(non_deep_gru_forward_min, non_deep_gru_backward_min)

non_deep_gru_forward_hour = GRU(input_size, hidden_size)
non_deep_gru_backward_hour = GRU(input_size, hidden_size)
bidirectional_gru_encoder_hour = BidirectionalRNNWithAttention(non_deep_gru_forward_hour, non_deep_gru_backward_hour)
#fc layer
non_deep_gru_fc_dst_min = DeepNeuralNetwork(2*hidden_size, pred_length, *architecture)
non_deep_gru_fc_kp_min = DeepNeuralNetwork(2*hidden_size, hour_to_3_hour(pred_length), *architecture)

non_deep_gru_fc_dst_hour = DeepNeuralNetwork(2*hidden_size, pred_length, *architecture)
non_deep_gru_fc_kp_hour = DeepNeuralNetwork(2*hidden_size, hour_to_3_hour(pred_length), *architecture)


In [ ]:
RefinedNonDeepGRU_min = to_device(RefinedArchitecture(bidirectional_gru_encoder_min, non_deep_gru_fc_dst_min, non_deep_gru_fc_kp_min), device)
RefinedNonDeepGRU_hour = to_device(RefinedArchitecture(bidirectional_gru_encoder_hour, non_deep_gru_fc_dst_hour, non_deep_gru_fc_kp_hour), device)

In [ ]:
RefinedNonDeepGRU_min_history = []

In [ ]:
RefinedNonDeepGRU_min_history += RefinedNonDeepGRU_min.fit(epochs, max_lr, train_mr_dl, test_mr_dl, weigth_decay, grad_clip, opt_func)

In [ ]:
RefinedNonDeepGRU_hour_history = []

In [ ]:
RefinedNonDeepGRU_hour_history += RefinedNonDeepGRU_hour.fit(epochs, max_lr, train_hr_dl, test_hr_dl, weigth_decay, grad_clip, opt_func)

Epoch [0]
	last_lr: 0.00004
	train_overall_loss: 1.3516
	train_main_loss: 1.3516
	train_output_loss: 0.0002
	train_encoder_loss: 0.0001
	val_overall_loss: 1.1860
	val_main_loss: 1.1860
	val_output_loss: 0.0003
	val_encoder_loss: 0.0001
Epoch [1]
	last_lr: 0.00005
	train_overall_loss: 0.9823
	train_main_loss: 0.9823
	train_output_loss: 0.0003
	train_encoder_loss: 0.0001
	val_overall_loss: 1.0192
	val_main_loss: 1.0192
	val_output_loss: 0.0005
	val_encoder_loss: 0.0001
Epoch [2]
	last_lr: 0.00006
	train_overall_loss: 0.8637
	train_main_loss: 0.8637
	train_output_loss: 0.0005
	train_encoder_loss: 0.0001
	val_overall_loss: 0.9084
	val_main_loss: 0.9084
	val_output_loss: 0.0005
	val_encoder_loss: 0.0001
Epoch [3]
	last_lr: 0.00008
	train_overall_loss: 0.7905
	train_main_loss: 0.7905
	train_output_loss: 0.0005
	train_encoder_loss: 0.0001
	val_overall_loss: 0.8263
	val_main_loss: 0.8263
	val_output_loss: 0.0005
	val_encoder_loss: 0.0001
Epoch [4]
	last_lr: 0.00010
	train_overall_loss: 0.7357


KeyboardInterrupt: 

#### RNN

In [ ]:
non_deep_rnn_forward_min = VanillaRNN(input_size, hidden_size)
non_deep_rnn_backward_min = VanillaRNN(input_size, hidden_size)
bidirectional_rnn_encoder_min = BidirectionalRNNWithAttention(non_deep_rnn_forward_min, non_deep_rnn_backward_min)

non_deep_rnn_forward_hour = VanillaRNN(input_size, hidden_size)
non_deep_rnn_backward_hour = VanillaRNN(input_size, hidden_size)
bidirectional_rnn_encoder_hour = BidirectionalRNNWithAttention(non_deep_rnn_forward_hour, non_deep_rnn_backward_hour)
#fc layer
non_deep_rnn_fc_dst_min = DeepNeuralNetwork(2*hidden_size, pred_length, *architecture)
non_deep_rnn_fc_kp_min = DeepNeuralNetwork(2*hidden_size, hour_to_3_hour(pred_length), *architecture)

non_deep_rnn_fc_dst_hour = DeepNeuralNetwork(2*hidden_size, pred_length, *architecture)
non_deep_rnn_fc_kp_hour = DeepNeuralNetwork(2*hidden_size, hour_to_3_hour(pred_length), *architecture)


In [ ]:
RefinedNonDeepVanillaRNN_min = to_device(RefinedArchitecture(bidirectional_rnn_encoder_min, non_deep_rnn_fc_dst_min, non_deep_rnn_fc_kp_min), device)
RefinedNonDeepVanillaRNN_hour = to_device(RefinedArchitecture(bidirectional_rnn_encoder_hour, non_deep_rnn_fc_dst_hour, non_deep_rnn_fc_kp_hour), device)

In [ ]:
RefinedNonDeepVanillaRNN_min_history = []

In [ ]:
RefinedNonDeepVanillaRNN_min_history += RefinedNonDeepVanillaRNN_min.fit(epochs, max_lr, train_mr_dl, test_mr_dl, weigth_decay, grad_clip, opt_func)

In [ ]:
RefinedNonDeepVanillaRNN_hour_history = []

In [ ]:
RefinedNonDeepVanillaRNN_hour_history += RefinedNonDeepVanillaRNN_hour.fit(epochs, max_lr, train_hr_dl, test_hr_dl, weigth_decay, grad_clip, opt_func)

Epoch [0]
	last_lr: 0.00004
	train_overall_loss: 1.2946
	train_main_loss: 1.2946
	train_output_loss: 0.0002
	train_encoder_loss: 0.0002
	val_overall_loss: 1.1871
	val_main_loss: 1.1871
	val_output_loss: 0.0003
	val_encoder_loss: 0.0002
Epoch [1]
	last_lr: 0.00005
	train_overall_loss: 1.0257
	train_main_loss: 1.0257
	train_output_loss: 0.0003
	train_encoder_loss: 0.0002
	val_overall_loss: 1.0831
	val_main_loss: 1.0830
	val_output_loss: 0.0005
	val_encoder_loss: 0.0002
Epoch [2]
	last_lr: 0.00006
	train_overall_loss: 0.9554
	train_main_loss: 0.9554
	train_output_loss: 0.0005
	train_encoder_loss: 0.0002
	val_overall_loss: 1.0201
	val_main_loss: 1.0201
	val_output_loss: 0.0006
	val_encoder_loss: 0.0002


KeyboardInterrupt: 

## Normal models with attention

### 1d Convolutional encoder

In [ ]:
## Architecture
hidden_size = 10
input_size = 20
##nn architecture
architecture = ()

In [ ]:
#encoders
cnn_encoder_min = Simple1DCNN(architecture, input_size, hidden_size)
cnn_encoder_hour = Simple1DCNN(architecture, input_size, hidden_size)
#fc layer
cnn_fc_dst_min = DeepNeuralNetwork(hidden_size, pred_length, *architecture)
cnn_fc_kp_min = DeepNeuralNetwork(hidden_size, hour_to_3_hour(pred_length), *architecture)

cnn_fc_dst_hour = DeepNeuralNetwork(hidden_size, pred_length, *architecture)
cnn_fc_kp_hour = DeepNeuralNetwork(hidden_size, hour_to_3_hour(pred_length), *architecture)

In [ ]:
NormalCNN_min = to_device(NormalArchitecture(cnn_encoder_min, cnn_fc_dst_min, cnn_fc_kp_min), device)
NormalCNN_hour = to_device(NormalArchitecture(cnn_encoder_hour, cnn_fc_dst_hour, cnn_fc_kp_hour), device)

In [ ]:
##hyperparameters
epochs = 100
max_lr = 1e-3
weigth_decay = 1e-4
grad_clip = 1e-2
#opt_func = Adam
opt_func = RMSprop

In [ ]:
NormalCNN_min_history = []

In [ ]:
NormalCNN_min_history += NormalCNN_min.fit(epochs, max_lr, train_mn_dl, test_mn_dl, weigth_decay, grad_clip, opt_func)

In [ ]:
NormalCNN_hour_history = []

In [ ]:
NormalCNN_hour_history += NormalCNN_hour.fit(epochs, max_lr, train_hn_dl, test_hn_dl, weigth_decay, grad_clip, opt_func)

RuntimeError: Given groups=1, weight of size [10, 20, 3], expected input[32, 10, 20] to have 20 channels, but got 10 channels instead

### Bidirectional Deep LSTM encoder

In [ ]:
## Architecture
hidden_size = 10
input_size = 20
##nn architecture
architecture = ()

In [ ]:
#encoders
deep_lstm_encoder_min_forward = DeepLSTM(hidden_size, input_size, architecture)
deep_lstm_encoder_min_backward = DeepLSTM(hidden_size, input_size, architecture)
deep_lstm_encoder_min = BidirectionalRNNWithAttention(deep_lstm_encoder_min_forward,deep_lstm_encoder_min_backward)

deep_lstm_encoder_hour_forward = DeepLSTM(hidden_size, input_size, architecture)
deep_lstm_encoder_hour_backward = DeepLSTM(hidden_size, input_size, architecture)
deep_lstm_encoder_hour = BidirectionalRNNWithAttention(deep_lstm_encoder_hour_forward,deep_lstm_encoder_hour_backward)

#fc layer
deep_lstm_fc_dst_min = DeepNeuralNetwork(2*hidden_size, pred_length, *architecture)
deep_lstm_fc_kp_min = DeepNeuralNetwork(2*hidden_size, hour_to_3_hour(pred_length), *architecture)

deep_lstm_fc_dst_hour = DeepNeuralNetwork(2*hidden_size, pred_length, *architecture)
deep_lstm_fc_kp_hour = DeepNeuralNetwork(2*hidden_size, hour_to_3_hour(pred_length), *architecture)

In [ ]:
NormalLSTM_min = to_device(NormalArchitecture(deep_lstm_encoder_min, deep_lstm_fc_dst_min, deep_lstm_fc_kp_min), device)
NormalLSTM_hour = to_device(NormalArchitecture(deep_lstm_encoder_hour, deep_lstm_fc_dst_hour, deep_lstm_fc_kp_hour), device)

In [ ]:
##hyperparameters
epochs = 100
max_lr = 1e-3
weigth_decay = 1e-4
grad_clip = 1e-2
#opt_func = Adam
opt_func = RMSprop

In [ ]:
NormalLSTM_min_history = []

In [ ]:
NormalLSTM_min_history += NormalLSTM_min.fit(epochs, max_lr, train_mn_dl, test_mn_dl, weigth_decay, grad_clip, opt_func)

In [ ]:
NormalLSTM_hour_history = []

In [ ]:
NormalLSTM_hour_history += NormalLSTM_hour.fit(epochs, max_lr, train_hn_dl, test_hn_dl, weigth_decay, grad_clip, opt_func)

Epoch [0]:
	last_lr: 0.00004
	train_loss: 1.3096
	val_loss: 0.9849
Epoch [1]:
	last_lr: 0.00005
	train_loss: 0.9692
	val_loss: 0.8660
Epoch [2]:
	last_lr: 0.00006
	train_loss: 0.8529
	val_loss: 0.7863
Epoch [3]:
	last_lr: 0.00008
	train_loss: 0.7619
	val_loss: 0.7326
Epoch [4]:
	last_lr: 0.00010
	train_loss: 0.7028
	val_loss: 0.6870
Epoch [5]:
	last_lr: 0.00013
	train_loss: 0.6619
	val_loss: 0.6704
Epoch [6]:
	last_lr: 0.00016
	train_loss: 0.6341
	val_loss: 0.6449
Epoch [7]:
	last_lr: 0.00020
	train_loss: 0.6039
	val_loss: 0.6322
Epoch [8]:
	last_lr: 0.00024
	train_loss: 0.5860
	val_loss: 0.6194
Epoch [9]:
	last_lr: 0.00028
	train_loss: 0.5695
	val_loss: 0.5994


KeyboardInterrupt: 

### Bidirectional Deep GRU encoder

In [ ]:
## Architecture
hidden_size = 10
input_size = 20
##nn architecture
architecture = ()

In [ ]:
#encoders

deep_gru_encoder_min_forward = DeepGRU(hidden_size, input_size, architecture)
deep_gru_encoder_min_backward = DeepGRU(hidden_size, input_size, architecture)
deep_gru_encoder_min = BidirectionalRNNWithAttention(deep_gru_encoder_min_forward,deep_gru_encoder_min_backward)

deep_gru_encoder_hour_forward = DeepGRU(hidden_size, input_size, architecture)
deep_gru_encoder_hour_backward = DeepGRU(hidden_size, input_size, architecture)
deep_gru_encoder_hour = BidirectionalRNNWithAttention(deep_gru_encoder_hour_forward,deep_gru_encoder_hour_backward)
#fc layer
deep_gru_fc_dst_min = DeepNeuralNetwork(2*hidden_size, pred_length, *architecture)
deep_gru_fc_kp_min = DeepNeuralNetwork(2*hidden_size,hour_to_3_hour(pred_length), *architecture)

deep_gru_fc_dst_hour = DeepNeuralNetwork(2*hidden_size, pred_length, *architecture)
deep_gru_fc_kp_hour = DeepNeuralNetwork(2*hidden_size, hour_to_3_hour(pred_length), *architecture)

In [ ]:
NormalGRU_min = to_device(NormalArchitecture(deep_gru_encoder_min, deep_gru_fc_dst_min, deep_gru_fc_kp_min), device)
NormalGRU_hour = to_device(NormalArchitecture(deep_gru_encoder_hour, deep_gru_fc_dst_hour, deep_gru_fc_kp_hour), device)

In [ ]:
##hyperparameters
epochs = 100
max_lr = 1e-3
weigth_decay = 1e-4
grad_clip = 1e-2
#opt_func = Adam
opt_func = RMSprop

In [ ]:
NormalGRU_min_history = []

In [ ]:
NormalGRU_min_history += NormalGRU_min.fit(epochs, max_lr, train_mn_dl, test_mn_dl, weigth_decay, grad_clip, opt_func)

RuntimeError: bad allocation

In [ ]:
NormalGRU_hour_history = []

In [ ]:
NormalGRU_hour_history += NormalGRU_hour.fit(epochs, max_lr, train_hn_dl, test_hn_dl, weigth_decay, grad_clip, opt_func)

Epoch [0]:
	last_lr: 0.00004
	train_loss: 0.9408
	val_loss: 0.8459


KeyboardInterrupt: 

### Bidirectional Deep Vanilla RNN encoder

In [ ]:
## Architecture
hidden_size = 10
input_size = 20
##nn architecture
architecture = ()

In [ ]:
#encoders
deep_rnn_encoder_min_forward = DeepVanillaRNN(hidden_size, input_size, architecture)
deep_rnn_encoder_min_backward = DeepVanillaRNN(hidden_size, input_size, architecture)
deep_rnn_encoder_min = BidirectionalRNNWithAttention(deep_rnn_encoder_min_forward, deep_rnn_encoder_min_backward)

deep_rnn_encoder_hour_forward = DeepVanillaRNN(hidden_size, input_size, architecture)
deep_rnn_encoder_hour_backward = DeepVanillaRNN(hidden_size, input_size, architecture)
deep_rnn_encoder_hour = BidirectionalRNNWithAttention(deep_rnn_encoder_hour_forward,deep_rnn_encoder_hour_backward)
#fc layer
deep_rnn_fc_dst_min = DeepNeuralNetwork(2*hidden_size, pred_length, *architecture)
deep_rnn_fc_kp_min = DeepNeuralNetwork(2*hidden_size, hour_to_3_hour(pred_length), *architecture)

deep_rnn_fc_dst_hour = DeepNeuralNetwork(2*hidden_size, pred_length, *architecture)
deep_rnn_fc_kp_hour = DeepNeuralNetwork(2*hidden_size, hour_to_3_hour(pred_length), *architecture)

In [ ]:
NormalVanillaRNN_min = to_device(NormalArchitecture(deep_rnn_encoder_min, deep_rnn_fc_dst_min, deep_rnn_fc_kp_min), device)
NormalVanillaRNN_hour = to_device(NormalArchitecture(deep_rnn_encoder_hour, deep_rnn_fc_dst_hour, deep_rnn_fc_kp_hour), device)

In [ ]:
##hyperparameters
epochs = 100
max_lr = 1e-3
weigth_decay = 1e-4
grad_clip = 1e-2
#opt_func = Adam
opt_func = RMSprop

In [ ]:
NormalVanillaRNN_min_history = []

In [ ]:
NormalVanillaRNN_min_history += NormalVanillaRNN_min.fit(epochs, max_lr, train_mn_dl, test_mn_dl, weigth_decay, grad_clip, opt_func)

In [ ]:
NormalVanillaRNN_hour_history = []

In [ ]:
NormalVanillaRNN_hour_history += NormalVanillaRNN_hour.fit(epochs, max_lr, train_hn_dl, test_hn_dl, weigth_decay, grad_clip, opt_func)

Epoch [0]:
	last_lr: 0.00004
	train_loss: 1.3040
	val_loss: 1.0009
Epoch [1]:
	last_lr: 0.00005
	train_loss: 1.0306
	val_loss: 0.9201
Epoch [2]:
	last_lr: 0.00006
	train_loss: 0.9532
	val_loss: 0.8801


KeyboardInterrupt: 

### Non deep bidirectional architectures with attention

#### LSTM

In [30]:
non_deep_lstm_forward_min = LSTM(input_size, hidden_size)
non_deep_lstm_backward_min = LSTM(input_size, hidden_size)
bidirectional_lstm_encoder_min = BidirectionalRNNWithAttention(non_deep_lstm_forward_min, non_deep_lstm_backward_min)

non_deep_lstm_forward_hour = LSTM(input_size, hidden_size)
non_deep_lstm_backward_hour = LSTM(input_size, hidden_size)
bidirectional_lstm_encoder_hour = BidirectionalRNNWithAttention(non_deep_lstm_forward_hour, non_deep_lstm_backward_hour)
#fc layer
non_deep_lstm_fc_dst_min = DeepNeuralNetwork(2*hidden_size, pred_length, *architecture)
non_deep_lstm_fc_kp_min = DeepNeuralNetwork(2*hidden_size, hour_to_3_hour(pred_length), *architecture)

non_deep_lstm_fc_dst_hour = DeepNeuralNetwork(2*hidden_size, pred_length, *architecture)
non_deep_lstm_fc_kp_hour = DeepNeuralNetwork(2*hidden_size, hour_to_3_hour(pred_length), *architecture)

In [31]:
NormalNonDeepLSTM_min = to_device(NormalArchitecture(bidirectional_lstm_encoder_min, non_deep_lstm_fc_dst_min, non_deep_lstm_fc_kp_min), device)
NormalNonDeepLSTM_hour = to_device(NormalArchitecture(bidirectional_lstm_encoder_hour, non_deep_lstm_fc_dst_hour, non_deep_lstm_fc_kp_hour), device)

In [ ]:
NormalNonDeepLSTM_min_history = []

In [ ]:
NormalNonDeepLSTM_min_history += NormalNonDeepLSTM_min.fit(epochs, max_lr, train_mn_dl, test_mn_dl, weigth_decay, grad_clip, opt_func)

In [32]:
NormalNonDeepLSTM_hour_history = []

In [33]:
NormalNonDeepLSTM_hour_history += NormalNonDeepLSTM_hour.fit(epochs, max_lr, train_hn_dl, test_hn_dl, weigth_decay, grad_clip, opt_func)

Epoch [0]:
	last_lr: 0.00005
	train_loss: 2.0350
	val_loss: 2.0597
Epoch [1]:
	last_lr: 0.00014
	train_loss: 2.0237
	val_loss: 2.0414
Epoch [2]:
	last_lr: 0.00026
	train_loss: 1.9437
	val_loss: 1.7768


KeyboardInterrupt: 

#### GRU

In [34]:
non_deep_gru_forward_min = GRU(input_size, hidden_size)
non_deep_gru_backward_min = GRU(input_size, hidden_size)
bidirectional_gru_encoder_min = BidirectionalRNNWithAttention(non_deep_gru_forward_min, non_deep_gru_backward_min)

non_deep_gru_forward_hour = GRU(input_size, hidden_size)
non_deep_gru_backward_hour = GRU(input_size, hidden_size)
bidirectional_gru_encoder_hour = BidirectionalRNNWithAttention(non_deep_gru_forward_hour, non_deep_gru_backward_hour)
#fc layer
non_deep_gru_fc_dst_min = DeepNeuralNetwork(2*hidden_size, pred_length, *architecture)
non_deep_gru_fc_kp_min = DeepNeuralNetwork(2*hidden_size, hour_to_3_hour(pred_length), *architecture)

non_deep_gru_fc_dst_hour = DeepNeuralNetwork(2*hidden_size, pred_length, *architecture)
non_deep_gru_fc_kp_hour = DeepNeuralNetwork(2*hidden_size, hour_to_3_hour(pred_length), *architecture)


In [35]:
NormalNonDeepGRU_min = to_device(NormalArchitecture(bidirectional_gru_encoder_min, non_deep_gru_fc_dst_min, non_deep_gru_fc_kp_min), device)
NormalNonDeepGRU_hour = to_device(NormalArchitecture(bidirectional_gru_encoder_hour, non_deep_gru_fc_dst_hour, non_deep_gru_fc_kp_hour), device)

In [ ]:
NormalNonDeepGRU_min_history = []

In [ ]:
NormalNonDeepGRU_min_history += NormalNonDeepGRU_min.fit(epochs, max_lr, train_mn_dl, test_mn_dl, weigth_decay, grad_clip, opt_func)

In [36]:
NormalNonDeepGRU_hour_history = []

In [37]:
NormalNonDeepGRU_hour_history += NormalNonDeepGRU_hour.fit(epochs, max_lr, train_hn_dl, test_hn_dl, weigth_decay, grad_clip, opt_func)

Epoch [0]:
	last_lr: 0.00005
	train_loss: 2.1161
	val_loss: 2.1215
Epoch [1]:
	last_lr: 0.00014
	train_loss: 2.0831
	val_loss: 2.0747


KeyboardInterrupt: 

#### RNN

In [38]:
non_deep_rnn_forward_min = VanillaRNN(input_size, hidden_size)
non_deep_rnn_backward_min = VanillaRNN(input_size, hidden_size)
bidirectional_rnn_encoder_min = BidirectionalRNNWithAttention(non_deep_rnn_forward_min, non_deep_rnn_backward_min)

non_deep_rnn_forward_hour = VanillaRNN(input_size, hidden_size)
non_deep_rnn_backward_hour = VanillaRNN(input_size, hidden_size)
bidirectional_rnn_encoder_hour = BidirectionalRNNWithAttention(non_deep_rnn_forward_hour, non_deep_rnn_backward_hour)
#fc layer
non_deep_rnn_fc_dst_min = DeepNeuralNetwork(2*hidden_size, pred_length, *architecture)
non_deep_rnn_fc_kp_min = DeepNeuralNetwork(2*hidden_size, hour_to_3_hour(pred_length), *architecture)

non_deep_rnn_fc_dst_hour = DeepNeuralNetwork(2*hidden_size, pred_length, *architecture)
non_deep_rnn_fc_kp_hour = DeepNeuralNetwork(2*hidden_size, hour_to_3_hour(pred_length), *architecture)


In [39]:
NormalNonDeepVanillaRNN_min = to_device(NormalArchitecture(bidirectional_rnn_encoder_min, non_deep_rnn_fc_dst_min, non_deep_rnn_fc_kp_min), device)
NormalNonDeepVanillaRNN_hour = to_device(NormalArchitecture(bidirectional_rnn_encoder_hour, non_deep_rnn_fc_dst_hour, non_deep_rnn_fc_kp_hour), device)

In [ ]:
NormalNonDeepVanillaRNN_min_history = []

In [ ]:
RefinedNonDeepVanillaRNN_min_history += NormalNonDeepVanillaRNN_min.fit(epochs, max_lr, train_mn_dl, test_mn_dl, weigth_decay, grad_clip, opt_func)

In [40]:
NormalNonDeepVanillaRNN_hour_history = []

In [41]:
NormalNonDeepVanillaRNN_hour_history += NormalNonDeepVanillaRNN_hour.fit(epochs, max_lr, train_hn_dl, test_hn_dl, weigth_decay, grad_clip, opt_func)

Epoch [0]:
	last_lr: 0.00005
	train_loss: 2.0527
	val_loss: 2.0624


KeyboardInterrupt: 

# Conclusion